In [4]:
from higlass.client import View, Track
from higlass.tilesets import cooler
import higlass


view1 = View([
    Track(track_type='top-axis', position='top'),
    Track(track_type='heatmap', position='center',
          tileset_uuid='CQMd6V_cRw6iCI_-Unl3PQ',
          server="http://higlass.io/api/v1/",
          height=250,
          options={ 'valueScaleMax': 0.5 }),
])

display, server, viewconf = higlass.display([view1])
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'JBpmtdGOTxms4yThORWOhA', 'tracks': {'top': [{'ty…